# Assignment 4 - 'Meteorite'
## Instructions

Open the PD patch first\
Run the cell below

In [2]:
import pandas as pd
import os
import time
from pythonosc import udp_client
pd.set_option('display.max_rows', None)

#specify filepath
file_path = "Meteorite_Landings.csv"

#read file
df = pd.read_csv(file_path)

#Sort by the 'id'
df_sorted = df.sort_values(by='id')

# Select rows where 'id' is between 366 and 466
df_filtered = df_sorted[(df_sorted['id'] >= 366) & (df_sorted['id'] <= 466)]

# List of columns to be removed
columns_to_remove = ['name', 'nametype', 'GeoLocation', 'mass (g)']

# Drop the specified columns
df_filtered = df_filtered.drop(columns=columns_to_remove)

# Normalize the 'reclat' column to be between 40 and 800
min_reclat = df_filtered['reclat'].min()
max_reclat = df_filtered['reclat'].max()
df_filtered['Frequency'] = 40 + ((df_filtered['reclat'] - min_reclat) / (max_reclat - min_reclat)) * (500 - 40)

# Drop the original 'reclat' column
df_filtered = df_filtered.drop(columns=['reclat'])

# Normalize the 'year' column to be between 1 and 100
min_year = df_filtered['year'].min()
max_year = df_filtered['year'].max()
df_filtered['Velocity'] = 1 + ((df_filtered['year'] - min_year) / (max_year - min_year)) * 99

# Drop the original 'year' column
df_filtered = df_filtered.drop(columns=['year'])

# Normalize the 'reclong' column to be between 50 and 3000

df_filtered['reclong'] = df_filtered['reclong'].fillna(0).astype(int)

min_reclong = df_filtered['reclong'].min()
max_reclong = df_filtered['reclong'].max()
df_filtered['Duration'] = 0.05 + ((df_filtered['reclong'] - min_reclong) / (max_reclong - min_reclong)) * (3 - 0.05)

# Drop the original 'reclong' column
df_filtered = df_filtered.drop(columns=['reclong'])

# Replace 'Found' with 1 and 'Fell' with 0 in the 'fall' column
df_filtered['fall'] = df_filtered['fall'].replace({'Found': 1, 'Fell': 0})

# Rename the 'fall' column to 'BinarySignal'
df_filtered = df_filtered.rename(columns={'fall': 'BinarySignal'})

#Custom function to alter 'recclass' column
def custom_recclass_mapping(value):
    if value.startswith('H'):
        return 0
    elif value.startswith('L'):
        return 1
    elif value.startswith('iron'):
        return 2
    else:
        return 3
df_filtered['recclass'] = df_filtered['recclass'].apply(custom_recclass_mapping)

# Rename the 'nametype' column to 'WaveShape'
df_filtered = df_filtered.rename(columns={'recclass': 'WaveShape'})

# Display the new DataFrame
print(df_filtered)


# Create an OSC client to send messages to Pure Data
pd_client = udp_client.SimpleUDPClient("localhost", 5005)  # Adjust host and port accordingly

# Function to send Transport messages to Pure Data
def send_pd_transport(parameter, value):
    address = f"/meteorite/{parameter}"  # Define OSC address pattern
    pd_client.send_message(address, value)
    
send_pd_transport("Transport", "Start")

#Function to send OSC messages to Pure Data
def send_pd_message(parameter, value):
    address = f"/meteorite/{parameter}"  # Define OSC address pattern
    pd_client.send_message(address, value)

#Send data to Pure Data
for index, row in df_filtered.iterrows():
    send_pd_message("Duration", row["Duration"])
    send_pd_message("Frequency", row["Frequency"])
    send_pd_message("Velocity", row["Velocity"])
    send_pd_message("BinarySignal", row["BinarySignal"])
    send_pd_message("WaveShape", row["WaveShape"])
    
    time.sleep(row["Duration"])
    
send_pd_transport("Transport", "Stop")

       id  WaveShape  BinarySignal   Frequency    Velocity  Duration
1477  366          0             1  369.545236   99.582278  1.655574
1478  367          0             1  369.713190   99.582278  1.655574
1479  368          1             1  370.074839   99.582278  1.665246
1515  369          0             1  412.551256   67.000000  0.659344
4     370          1             0  158.483072   57.810127  1.007541
1517  371          0             1  391.153452   76.607595  0.620656
1518  372          3             1  396.087841   82.873418  0.591639
1519  373          3             1  376.118139   90.810127  0.659344
1520  374          3             1  402.531556   49.873418  1.916721
1522  375          0             1  413.213043   68.670886  0.630328
1523  376          0             1  343.086817   82.455696  2.119836
1524  377          3             1         NaN   87.050633  1.626557
1525  378          1             1   40.000000   61.987342  3.000000
5     379          3             0

KeyboardInterrupt: 